<a href="https://colab.research.google.com/github/anishmiddela/Deep-Learning/blob/main/CNN_Alexnet_FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import torch
import torchvision
import torch.nn as tn
import torch.optim as opt
import torchvision.transforms as tf

In [ ]:
# Define device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define transform for data
transformTrain = tf.Compose([
    tf.RandomResizedCrop(224),
    tf.RandomHorizontalFlip(),
    tf.ToTensor(),
    tf.Normalize((0.5,), (0.5,))
])

In [ ]:
# Define transform for testing data
transformTest = tf.Compose([
    tf.Resize(256),
    tf.CenterCrop(224),
    tf.ToTensor(),
    tf.Normalize((0.5,), (0.5,))
])

In [ ]:
# Load FashionMNIST dataset
training_set = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transformTrain)
trainloader = torch.utils.data.DataLoader(training_set, batch_size=128,
                                          shuffle=True, num_workers=2)
testing_set = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transformTest)
testloader = torch.utils.data.DataLoader(testing_set, batch_size=128,
                                         shuffle=False, num_workers=2)

In [ ]:
len(training_set)

60000

In [ ]:
len(testing_set)

10000

In [ ]:
# AlexNet architecture
class AlexNet(tn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = tn.Sequential(
            tn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            tn.ReLU(inplace=True),
            tn.MaxPool2d(kernel_size=3, stride=2),
            tn.Conv2d(64, 192, kernel_size=5, padding=2),
            tn.ReLU(inplace=True),
            tn.MaxPool2d(kernel_size=3, stride=2),
            tn.Conv2d(192, 384, kernel_size=3, padding=1),
            tn.ReLU(inplace=True),
            tn.Conv2d(384, 256, kernel_size=3, padding=1),
            tn.ReLU(inplace=True),
            tn.Conv2d(256, 256, kernel_size=3, padding=1),
            tn.ReLU(inplace=True),
            tn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = tn.AdaptiveAvgPool2d((6, 6))
        self.classifier = tn.Sequential(
            tn.Dropout(),
            tn.Linear(256 * 6 * 6, 4096),
            tn.ReLU(inplace=True),
            tn.Dropout(),
            tn.Linear(4096, 4096),
            tn.ReLU(inplace=True),
            tn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
# Define model, loss function, and optimizer
alex = AlexNet().to(device)
optimize = opt.SGD(alex.parameters(), lr=0.001, momentum=0.9)
lossfunc = tn.CrossEntropyLoss()

In [ ]:
# Train the network
for epoch in range(18):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        # Get inputs and labels
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimize.zero_grad()

        # Forward + backward + optimize
        outputs = alex(inputs)
        loss = lossfunc(outputs, labels)
        loss.backward()
        optimize.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,   100] loss: 0.923
[1,   200] loss: 0.895
[1,   300] loss: 0.909
[1,   400] loss: 0.901
[2,   100] loss: 0.887
[2,   200] loss: 0.891
[2,   300] loss: 0.882
[2,   400] loss: 0.889
[3,   100] loss: 0.888
[3,   200] loss: 0.887
[3,   300] loss: 0.879
[3,   400] loss: 0.866
[4,   100] loss: 0.865
[4,   200] loss: 0.857
[4,   300] loss: 0.849
[4,   400] loss: 0.850
[5,   100] loss: 0.860
[5,   200] loss: 0.833
[5,   300] loss: 0.859
[5,   400] loss: 0.832
[6,   100] loss: 0.823
[6,   200] loss: 0.834
[6,   300] loss: 0.828
[6,   400] loss: 0.811
[7,   100] loss: 0.829
[7,   200] loss: 0.818
[7,   300] loss: 0.811
[7,   400] loss: 0.808
[8,   100] loss: 0.792
[8,   200] loss: 0.801
[8,   300] loss: 0.802
[8,   400] loss: 0.807
[9,   100] loss: 0.798
[9,   200] loss: 0.794
[9,   300] loss: 0.790
[9,   400] loss: 0.780
[10,   100] loss: 0.783
[10,   200] loss: 0.779
[10,   300] loss: 0.782
[10,   400] loss: 0.790
[11,   100] loss: 0.766
[11,   200] loss: 0.780
[11,   300] loss: 0.785
[11,

In [ ]:
# Test the network
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = alex(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 83 %
